# **Task 4**

## **Import Libraries**

In [ ]:
import kagglehub
import os
import pandas as pd

In [ ]:
!pip install -q spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install -q spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md

import spacy
from spacy import displacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 22.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from IPython.display import display, HTML

## **Load the data and Exploration**

In [ ]:
path = kagglehub.dataset_download("alaakhaled/conll003-englishversion")

print("Path to dataset files:", path)
print("Files in directory:", os.listdir(path))


100%|██████████| 960k/960k [00:00<00:00, 36.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/alaakhaled/conll003-englishversion/versions/1
Files in directory: ['train.txt', 'test.txt', 'metadata', 'valid.txt']


In [ ]:
# Load CoNLL-2003 train set
file_path = os.path.join(path, 'train.txt')

In [ ]:
# Parse CoNLL-2003 file into entity-label format
data = []
with open(file_path, 'r') as f:
    sentence = []
    for line in f:
        line = line.strip()
        if line:
            parts = line.split()
            if len(parts) == 4:
                word, pos, chunk, ner = parts
                sentence.append({"entity": word, "label": ner})
        else:
            if sentence:
                data.append(sentence)
                sentence = []
    if sentence:
        data.append(sentence)

print(f"\nLoaded {len(data)} sentences.")
print("Example sentence:", data[0][:10])  # Show first 10 entity-label pairs



Loaded 14987 sentences.
Example sentence: [{'entity': '-DOCSTART-', 'label': 'O'}]


## **Rule-based NER**

In [ ]:
def rule_based_ner(sentence):
    entities = []
    for i, (word, ner) in enumerate(sentence):
        # Rule: Capitalized words (not first in sentence) are entities
        if word[0].isupper() and i != 0:
            entities.append((word, "POTENTIAL_ENTITY"))
    return entities

In [ ]:
processed_data = []
for sentence in data:
    processed_sentence = [(item["entity"], item["label"]) for item in sentence]
    processed_data.append(processed_sentence)

In [ ]:
for sent in processed_data[:3]:
    print("Sentence:", sent)
    print("Rule-based entities:", rule_based_ner(sent))
    print()

Sentence: [('-DOCSTART-', 'O')]
Rule-based entities: []

Sentence: [('EU', 'B-ORG'), ('rejects', 'O'), ('German', 'B-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'B-MISC'), ('lamb', 'O'), ('.', 'O')]
Rule-based entities: [('German', 'POTENTIAL_ENTITY'), ('British', 'POTENTIAL_ENTITY')]

Sentence: [('Peter', 'B-PER'), ('Blackburn', 'I-PER')]
Rule-based entities: [('Blackburn', 'POTENTIAL_ENTITY')]



## **Model-based NER (spaCy)**

In [ ]:
# Function to perform model-based NER
def model_based_ner(sentence_tokens):
    text = " ".join([word for word, tag in sentence_tokens])
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

# Apply model-based NER to the first few processed sentences
print("Model-based NER results for the first 5 sentences:")
for i, sent in enumerate(processed_data[:5]):
    model_entities = model_based_ner(sent)
    print(f"Sentence {i+1}: {model_entities}")

Model-based NER results for the first 5 sentences:
Sentence 1: []
Sentence 2: [('EU', 'ORG'), ('German', 'NORP'), ('British', 'NORP')]
Sentence 3: [('Peter Blackburn', 'PERSON')]
Sentence 4: [('BRUSSELS', 'GPE'), ('1996-08-22', 'DATE')]
Sentence 5: [('The European Commission', 'ORG'), ('Thursday', 'DATE'), ('German', 'NORP'), ('British', 'NORP')]


## **Highlight and categorize entities**

In [ ]:
def highlight_entities(sentence_tokens, entities):
    text = " ".join([word for word, tag in sentence_tokens])
    html_text = text

    # Sort entities by their order in text
    sorted_entities = sorted(entities, key=lambda x: text.find(x[0]))

    for entity_text, entity_type in sorted_entities:
        html_text = html_text.replace(
            entity_text,
            f'<mark style="background-color: #FFD700" data-entity="{entity_type}">'
            f'{entity_text} ({entity_type})</mark>'
        )
    return html_text

# Example of highlighting using model-based NER results for the first 5 sentences
print("Highlighted entities using Model-based NER:")
for i, sent in enumerate(processed_data[:5]):
    model_entities = model_based_ner(sent)
    highlighted_html = highlight_entities(sent, model_entities)
    print(f"Sentence {i+1}:")
    display(HTML(highlighted_html))

Highlighted entities using Model-based NER:
Sentence 1:


Sentence 2:


Sentence 3:


Sentence 4:


Sentence 5:


# **Bouns Task 4**

In [ ]:
# Load two spaCy models
nlp_sm = spacy.load("en_core_web_sm")
nlp_md = spacy.load("en_core_web_md")

In [ ]:
def model_based_ner(sentence_tokens, nlp_model):
    text = " ".join([word for word, tag in sentence_tokens])
    doc = nlp_model(text)
    return doc

In [ ]:
# Pick first 2 sentences for visualization
sample_sentences = data[:2]

print("\nModel-based NER comparison:")

for i, sent in enumerate(sample_sentences):
    print(f"\n----- Sentence {i+1} -----")
    text = " ".join([w for w, t in sent])
    print("Text:", text)

    # Small model
    doc_sm = model_based_ner(sent, nlp_sm)
    print("Entities (sm):", [(ent.text, ent.label_) for ent in doc_sm.ents])

    # Medium model
    doc_md = model_based_ner(sent, nlp_md)
    print("Entities (md):", [(ent.text, ent.label_) for ent in doc_md.ents])

    # Visualize with displaCy
    print("\nVisualization (sm):")
    displacy.render(doc_sm, style="ent", jupyter=True)
    print("\nVisualization (md):")
    displacy.render(doc_md, style="ent", jupyter=True)


Model-based NER comparison:

----- Sentence 1 -----
Text: entity
Entities (sm): []
Entities (md): []

Visualization (sm):


/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Visualization (md):



----- Sentence 2 -----
Text: entity entity entity entity entity entity entity entity entity
Entities (sm): []
Entities (md): []

Visualization (sm):



Visualization (md):
